# Sumo Routes Sanity Check
## Introduction
This notebook runs Sumo using TraCI. All it does is send a vehicle into the network for each possible airport route (for example, North Toll Plaza to Terminal A lower curb to South Exit, etc). It records the complete route that Sumo sent that vehicle on, and the vehicle speed for each step of the simulation. This data can be visualized in the corresponding Observable Notebook, and used to sanity check the Sumo network file. For example, you may notice Sumo routing vehicles on service roads or on circuitious routes or through strange speed fluctuations. These behaviors can likely be traced to problems in the network file: Incorrect road classifications, speeds, or junctions.

## Code Overview
Some of the guts of the code has been put into traciHelper.py. The main thing you will likely want to change is the configuration file path (to change what network file the simulation will run with). You may also want to change what routes will be run and recorded.

NOTE: For some reason traci.start will always have to retry once to begin working. You will always see a " Retrying in 1 seconds" line printed to the output panel at the start of a simulation. You can ignore it.

## Input Parameters
The only thing you should need to adjust for a new run is the path to the sumo configuration file, `configFile`. You will likely also want to specify a new results file location as well, `resultsFile`. Using a path to an existing file will __overwrite__ old results.

The code within traciHelper.py does make some assumptions as to the Sumo network edge names. If your new network file has new edge names for the toll plaza, curb, or exit edges then you will need to make adjustments within traciHelper.py.

## Output Format
Output will be saved to `resultsFile` in json format. The results are a dictionary. The keys to the dictionary are the route names (as generated by traciHelper.py). Route names are generally in the format _start_ To _middle_ To _end_. The values in the dictionary are an ordered array of time step statistics. For each time step for which the vehicle was in the simulation, we recorded the latitude, longitude, speed, and edge speed limit.

## Example
Imagine that there was only one route we were assessing (there's normally upwards of 50). Then the results dictionary would look something like:
```python
{
       "RouteStartToRouteMiddleToRouteEnd": [[32.86552951992726, -97.03972693649231, 10, 12],
                                             [32.86556558448741, -97.03972482521979, 12, 12],
                                             [32.865616915820965, -97.03972182020354, 15, 15],
                                             [32.86568693886412, -97.0397177209373, 15, 15]]
}
```
The speeds are in __meters per second__. In this example you can see that the route took the test vehicle four time steps (also a huge underestimation). In the first time step it was driving under the speed limit of 12 meters/second, but for the other three time steps it was able to meet the speed limit.

## Set various paths here.
* Set the absolute or relative path to your sumocfg file.
* Set the absolute or relative path to where you would like the results saved.
* Set the absolute or relative path to where you would like the sumo log file to be written.

In [18]:
from os import path
import sys

configFile = "../Networks/20190909/config.sumocfg"
resultsFile = "../Results/RouteAnalysisFor20190909.json" # json format
logFile = "logfile_SumoRoutesSanityCheck.txt" # Errors in the Sumo simulation will be printed to here.

if (path.exists(resultsFile)):
    print("WARNING: Results file <" + resultsFile + "> already exists: Running the simulation will overwrite this file", file=sys.stderr)

## Run the following cell to actually run the simulation and assess routes. It should run in less than a minute.

In [2]:
import traciHelper # MUST import traciHelper before traci, otherwise it will complain it cannot find module traci
import traci
import traci.constants as tc
import numpy as np
import time
import uuid
import json
import imp
imp.reload(traciHelper)

startTime = time.time()
# We set the speeddev to 0 to keep the vehicle speeds on the route consistent and thus comparable.
# If we don't set this to 0 the vehicles will have a random speed deviation and will sometimes just
# drive under the speed limit. This isn't a problem with the network, so we want to eliminate such
# variance for this sanity check.
sumoCmd = [traciHelper.sumoBinary(), "-c", configFile, "--log", logFile, "--default.speeddev", "0"]
traci.start(sumoCmd)
print("Started TraCI")
routesDictionary = {}

registeredRoutes = traciHelper.registerRoutes()
print("Registered the routes")
for routeName in registeredRoutes:
    print("Assessing route '" + routeName + "'")
    vehicleName = str(uuid.uuid1())
    traci.vehicle.add(vehicleName, routeName)
    routesDictionary[routeName] = []
    # Run the simulation until this vehicle has completed it's route
    traci.simulationStep()
    while (vehicleName in traci.vehicle.getIDList()):
        vehicleStats = traciHelper.pollVehicle(vehicleName)
        routesDictionary[routeName].append(vehicleStats)
        traci.simulationStep()
traci.close()
print("Closed TraCI")
print("Running the simulation took: " + str(round(time.time() - startTime , 2)) + " seconds")

# Write out the results in json format
with open(resultsFile, 'w') as fp:
    json.dump(routesDictionary, fp)
print("Wrote results to " + resultsFile)

 Retrying in 1 seconds
Started TraCI
Registered the routes
Assessing route 'South_PlazaToA_top_1ToSouth_Exit'
Assessing route 'South_PlazaToA_top_2ToSouth_Exit'
Assessing route 'South_PlazaToA_bot_1ToSouth_Exit'
Assessing route 'South_PlazaToA_bot_2ToSouth_Exit'
Assessing route 'South_PlazaToB_top_1ToSouth_Exit'
Assessing route 'South_PlazaToB_top_2ToSouth_Exit'
Assessing route 'South_PlazaToB_top_3ToSouth_Exit'
Assessing route 'South_PlazaToC_top_1ToSouth_Exit'
Assessing route 'South_PlazaToC_top_2ToSouth_Exit'
Assessing route 'South_PlazaToC_top_3ToSouth_Exit'
Assessing route 'South_PlazaToC_bot_1ToSouth_Exit'
Assessing route 'South_PlazaToC_bot_2ToSouth_Exit'
Assessing route 'South_PlazaToC_bot_3ToSouth_Exit'
Assessing route 'South_PlazaToD_top_1ToSouth_Exit'
Assessing route 'South_PlazaToD_bot_1ToSouth_Exit'
Assessing route 'South_PlazaToE_top_1ToSouth_Exit'
Assessing route 'South_PlazaToE_top_2ToSouth_Exit'
Assessing route 'South_PlazaToE_bot_1ToSouth_Exit'
Assessing route 'South_